In [1]:
import pandas as pd                       #data processing
from sklearn.feature_extraction.text import TfidfVectorizer         #counting frequency of the word                  
from sklearn.model_selection import train_test_split               #dividing training and testing data 
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score      
from sklearn.svm import SVC
import re          #regular expression

In [2]:
def cleaner(impure_data):
    temp_list = []
    for item in impure_data:
        #finding words which start with @
        item = re.sub('@\S+', '', item)
        
        #finding words which start with http
        item = re.sub('http\S+\s*', '', item)
        
        #finding special characters, but not "emoji"
        item = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', item)
        temp_list.append(item)
    return temp_list

In [6]:
#Function to predict tweet sentiment
def tweet_sentiment():
    #reading the tweets from csv files
    df = pd.read_csv("airlinetweets.csv")
    tweets = df["text"]
    polarity = df["airline_sentiment"].tolist()
    
    #cleaning tweets i.e. removing @mentions, http(s) links and special characters such as punctuations
    clean_tweet = cleaner(tweets)
    
    #initializing tf-idf vectorizer
    tf_idfvectorizer = TfidfVectorizer(sublinear_tf=True, use_idf=True)
    
    #splitting the data into train and test dataset in 70 : 30 ratio at random
    X_train, X_test, Y_train, Y_test = train_test_split(clean_tweet, polarity, test_size = 0.2)
    
    #vectorizing the training data
    #fit_transform() does two jobs, fit() and transform()
    #fit calculates the statistics of the data
    #transform takes care of any missing values or unexpected values by utilizing statistics calculated by fit
    train_corpus_tf_idf = tf_idfvectorizer.fit_transform(X_train) 
        
    #vectorizing the testing data
    #transform takes care of any missing values or unexpected values based on fit for training data
    test_corpus_tf_idf = tf_idfvectorizer.transform(X_test)
    
    #using SVC package to initialize a classifier with Linear kernel and other default parameters
    SVM_L = SVC(kernel= 'linear')
    
    #fitting the sparse matrix in the classifier with their respective sentiments
    SVM_L.fit(train_corpus_tf_idf, Y_train)
    
    #predicting the sentiments for the test dataset
    Y_pred = SVM_L.predict(test_corpus_tf_idf)
    
    #this print accuracy score for the test dataset
    print("Accuracy",accuracy_score(Y_test,Y_pred))
    
    
    #saving the data into a csv file in the current folder
    temp_df = pd.DataFrame()
    temp_df["Tweet"] = X_test
    temp_df["Sentiment"] = Y_test
    temp_df["Predicted Sentiment"] = Y_pred
    temp_df.to_csv("predicted_airlinetweets.csv")

    return(tf_idfvectorizer,SVM_L)

In [7]:
vectorizer,model = tweet_sentiment()

Accuracy 0.8084016393442623


In [11]:
vector = vectorizer.transform(["not bad journey."])
sentiment = model.predict(vector)
print(sentiment)

['negative']
